![alt text](./Cerny_logo_1.jpg)

# Analysis of Cerny ventilation recordings

#### Manual revision and trimming of ventilator recordings.

This notebook imports the preprocessed **Fabian ventilator parameters** data from pickle archive and performs trimming based on manual inspection of data.

The data processed and analysed in this Notebook were collected by the **Neonatal Emergency and Transport Service of the Peter Cerny Foundation**, Budapest, Hungary

**Author: Dr Gusztav Belteki**

### 1. Import the required libraries and set options

In [ ]:
import IPython
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import os
import sys
import pickle

from pandas import Series, DataFrame
from datetime import datetime, timedelta
from matplotlib import dates

%matplotlib inline
matplotlib.style.use('classic')
matplotlib.rcParams['figure.facecolor'] = 'w'

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)
# pd.set_option('mode.chained_assignment', None) 

import warnings
warnings.simplefilter("ignore")

In [ ]:
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("NumPy version: {}".format(np.__version__))
print("IPython version: {}".format(IPython.__version__))

### 2. List and set the working directory and the directory to write out data

In [ ]:
# Name of the external hard drive
DRIVE = 'GUSZTI'

# Directory on external drive to read the clinical from
DIR_READ = os.path.join(os.sep, 'Volumes', DRIVE, 'Fabian_new', 'fabian_patient_data_all_new')

# Path to project folder containing ventilation research results
PATH = os.path.join(os.sep, 'Users', 'guszti', 'Library', 'Mobile Documents', 'com~apple~CloudDocs', 
                            'Documents', 'Research', 'Ventilation')

# Folder to export the result of analysis
DIR_WRITE = os.path.join(PATH, 'ventilation_fabian_new', 'Analyses')
os.makedirs(DIR_WRITE, exist_ok = True)

# Folder on a USB stick to export data to and to import processed data exported by other Notebooks
DATA_DUMP = os.path.join(os.sep, '/Volumes', DRIVE, 'data_dump', 'fabian_new',)
os.makedirs(DATA_DUMP, exist_ok = True)

In [ ]:
DIR_READ, DIR_WRITE, DATA_DUMP

### 3. Import pickle archives

In [ ]:
with open(os.path.join(DATA_DUMP, 'clin_df_new.pickle'), 'rb') as handle:
    clin_df = pickle.load(handle)

with open(os.path.join(DATA_DUMP, 'data_pars_measurements_new_1_1305.pickle'), 'rb') as handle:
    data_pars_measurements = pickle.load(handle)
    
with open(os.path.join(DATA_DUMP, 'data_pars_settings_new_1_1305.pickle'), 'rb') as handle:
    data_pars_settings = pickle.load(handle)
    
with open(os.path.join(DATA_DUMP, 'data_pars_alarms_new_1_1305.pickle'), 'rb') as handle:
    data_pars_alarms = pickle.load(handle)

In [ ]:
len(data_pars_measurements)

In [ ]:
cases = sorted(data_pars_measurements.keys())

In [ ]:
# Import DataFrame with ventilation modes before trimming

with open(os.path.join(DATA_DUMP, 'vent_modes_unedited_new_1_1305.pickle'), 'rb') as handle:
    vent_modes_unedited = pickle.load(handle)

len(vent_modes_unedited)

### 4. Remove some abnormal recordings 

In [ ]:
# Remove recordings do not actually contain periods with the patient connected
to_remove = []

for case in to_remove:
    if case in cases:
        cases.remove(case)

### 5. Remove the periods from the beginning and the end of the recordings when the patient was not connected to the ventilator

This requires manual inspection of the tidal volume and pressure graphs

This dictionary contains tuples of the start and end points as strings
This was obtained by manual inspection of VTmand and PIP and the recordings
and manually removing the start and the end when the baby was not on the ventilator (e.g. no VTmand)

In [ ]:
with open(os.path.join(DATA_DUMP, 'limit_new_1_1305.pickle'), 'rb') as handle:
    limit = pickle.load(handle)

In [ ]:
len(limit)

In [ ]:
print(sorted(set(cases) - set(limit)))

In [ ]:
cases;

In [ ]:
case = sorted(set(cases) - set(limit))[0]

index = cases.index(case)
index

In [ ]:
vent_modes_unedited.loc[case]

In [ ]:
mapping_vent_mode = {1: 'IPPV', 2: 'SIPPV', 3: 'SIMV', 4: 'SIMVPSV', 5: 'PSV', 
                     6: 'CPAP', 7: 'NCPAP', 8: 'DUOPAP', 9: 'HFO', 10: 'O2therapy'}
mapping_vent_mode_rev = {value:key for key, value in mapping_vent_mode.items()}
mapping_vent_mode_rev

mapping_VG = {11: 'off', 12: 'on'}
mapping_VG_rev = {value:key for key, value in mapping_VG.items()}
mapping_VG_rev

par1 = 'Ventilator_mode'
par2 = 'VG_state'
dim = 'cmH$_2$O'

yticks = np.arange(1, 13)
yticklabels = ['IPPV', 'SIPPV','SIMV','SIMVPSV', 'PSV', 'CPAP', 'NCPAP', 'DUOPAP', 'HFO', 'O2therapy',
              'VG OFF', 'VG ON']

fig = plt.figure()
fig.set_size_inches(8, 4)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.1)
ax = fig.add_subplot(1, 1, 1)
        
data = data_pars_settings[case][par1].replace(mapping_vent_mode_rev)
data.plot(ax = ax, label = par1, color='blue', x_compat = True)
        
data_2 = data_pars_settings[case][par2].replace(mapping_VG_rev)
data_2.plot(ax = ax, label = par2, color='blue', linestyle='dashed', x_compat = True)
        
ax.set_xlabel('Time', size = 14, color = 'black')
ax.set_ylabel('', size = 14, color = 'black')
ax.set_ylim(0,17)
ax.set_yticks(yticks)
ax.set_yticklabels(yticklabels)
ax.set_title(case,  size = 14, color = 'black')
ax.legend(ncols=2)
ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
majorFmt = dates.DateFormatter('%H:%M')
ax.xaxis.set_major_formatter(majorFmt)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
ax.tick_params(which = 'both', labelsize=12)


In [ ]:
fig = plt.figure()
fig.set_size_inches(10, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)

ax0 = fig.add_subplot(2, 1, 1)
data_pars_measurements[case]['VTemand_kg'].plot(x_compat = True)
ax0.get_xaxis().set_visible(True)
ax0.set_ylabel('mL/kg', size = 12, color = 'black')
ax0.set_title(case,  size = 12, color = 'black')
ax0.legend(['VTemand_kg'], fontsize = 10, loc = 'best')
ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
majorFmt = dates.DateFormatter('%H:%M')
ax0.xaxis.set_major_formatter(majorFmt)
ax0.tick_params(which = 'both', labelsize=12)

ax1 = fig.add_subplot(2, 1, 2)
data_pars_measurements[case]['PIP'].plot(color = 'red', x_compat = True)
data_pars_measurements[case]['MAP'].plot(color = 'blue', x_compat = True)
data_pars_measurements[case]['PEEP'].plot(color = 'green', x_compat = True)
ax1.set_xlabel('Time', size = 12, color = 'black')
ax1.set_ylabel('cmH2O)', size = 12, color = 'black')
ax1.legend(['PIP','MAP', 'PEEP'], fontsize = 10, loc = 'best', ncol=3)
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
majorFmt = dates.DateFormatter('%H:%M')
ax1.xaxis.set_major_formatter(majorFmt)
ax1.tick_params(which = 'both', labelsize=12)

In [ ]:
fig = plt.figure()
fig.set_size_inches(10, 4)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)

ax1 = fig.add_subplot(1, 1, 1)
data_pars_measurements[case]['MAP'].plot(color = 'green', x_compat = True)
ax1.set_xlabel('Time', size = 12, color = 'black')
ax1.set_ylabel('cmH2O)', size = 12, color = 'black')
ax1.legend(['MAP', 'PEEP'], fontsize = 10, loc = 'best', ncol=3)
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
majorFmt = dates.DateFormatter('%H:%M')
ax1.xaxis.set_major_formatter(majorFmt)
ax1.tick_params(which = 'both', labelsize=12)

In [ ]:
limit[case] = eval("('%s', '%s')" % (data_pars_measurements[case].index[0], data_pars_measurements[case].index[-1]))
limit[case]

In [ ]:
limit[case] =  ('2024-05-06 23:23:45', '2024-05-07 00:25:00')

In [ ]:
fig = plt.figure()
fig.set_size_inches(10, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)

ax0 = fig.add_subplot(2, 1, 1)
data_pars_measurements[case]['VTemand_kg'][limit[case][0] : limit[case][1]].plot(x_compat = True)
ax0.get_xaxis().set_visible(False)
ax0.set_ylabel('mL/kg', size = 12, color = 'black')
ax0.set_title(case,  size = 12, color = 'black')
ax0.legend(['VTemand_kg'], fontsize = 10, loc = 'best')
ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
majorFmt = dates.DateFormatter('%H:%M')
ax0.xaxis.set_major_formatter(majorFmt)
ax0.tick_params(which = 'both', labelsize=12)

ax1 = fig.add_subplot(2, 1, 2)
data_pars_measurements[case]['PIP'][limit[case][0] : limit[case][1]].plot(color = 'red', x_compat = True)
data_pars_measurements[case]['MAP'][limit[case][0] : limit[case][1]].plot(color = 'blue', x_compat = True)
data_pars_measurements[case]['PEEP'][limit[case][0] : limit[case][1]].plot(color = 'green', x_compat = True)
ax1.set_xlabel('Time', size = 12, color = 'black')
ax1.set_ylabel('cmH2O)', size = 12, color = 'black')
ax1.legend(['PIP','MAP', 'PEEP'], fontsize = 10, loc = 'best', ncol=3)
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
majorFmt = dates.DateFormatter('%H:%M')
ax1.xaxis.set_major_formatter(majorFmt)
ax1.tick_params(which = 'both', labelsize=12)

In [ ]:
fig = plt.figure()
fig.set_size_inches(10, 4)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)

ax1 = fig.add_subplot(1, 1, 1)
data_pars_measurements[case]['MAP'][limit[case][0] : limit[case][1]].plot(color = 'green', x_compat = True)
ax1.set_xlabel('Time', size = 12, color = 'black')
ax1.set_ylabel('cmH2O)', size = 12, color = 'black')
ax1.legend(['MAP', 'PEEP'], fontsize = 10, loc = 'best', ncol=3)
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
majorFmt = dates.DateFormatter('%H:%M')
ax1.xaxis.set_major_formatter(majorFmt)
ax1.tick_params(which = 'both', labelsize=12)

In [ ]:
fig = plt.figure()
fig.set_size_inches(10, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)

ax0 = fig.add_subplot(2, 1, 1)
data_pars_measurements[case]['VThf_emand_kg'].plot()
#ax0.get_xaxis().set_visible(False)
ax0.set_ylabel('mL/kg', size = 12, color = 'black')
ax0.set_title(case,  size = 12, color = 'black')
ax0.legend(['VThf_emand_kg'], fontsize = 10, loc = 'best')
ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
majorFmt = dates.DateFormatter('%H:%M')
ax0.xaxis.set_major_formatter(majorFmt)
ax0.tick_params(which = 'both', labelsize=12)

ax1 = fig.add_subplot(2, 1, 2)
data_pars_measurements[case]['deltaP'].plot(color = 'red')
ax1.set_xlabel('Time', size = 12, color = 'black')
ax1.set_ylabel('cmH2O)', size = 12, color = 'black')

ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
majorFmt = dates.DateFormatter('%H:%M')
ax1.xaxis.set_major_formatter(majorFmt)
ax1.tick_params(which = 'both', labelsize=12)
ax1.set_ylim(0,70)

In [ ]:
fig = plt.figure()
fig.set_size_inches(10, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.05)

ax0 = fig.add_subplot(2, 1, 1)
data_pars_measurements[case]['VThf_emand_kg'][limit[case][0] : limit[case][1]].plot()
#ax0.get_xaxis().set_visible(False)
ax0.set_ylabel('mL/kg', size = 12, color = 'black')
ax0.set_title(case,  size = 12, color = 'black')
ax0.legend(['VThf_kg'], fontsize = 10, loc = 'best')
ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
majorFmt = dates.DateFormatter('%H:%M')
ax0.xaxis.set_major_formatter(majorFmt)
ax0.tick_params(which = 'both', labelsize=12)

ax1 = fig.add_subplot(2, 1, 2)
data_pars_measurements[case]['deltaP'][limit[case][0] : limit[case][1]].plot(color = 'red')
ax1.set_xlabel('Time', size = 12, color = 'black')
ax1.set_ylabel('cmH2O)', size = 12, color = 'black')
ax1.legend(['deltaP'], fontsize = 10, loc = 'best')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
majorFmt = dates.DateFormatter('%H:%M')
ax1.xaxis.set_major_formatter(majorFmt)
ax1.tick_params(which = 'both', labelsize=12)

In [ ]:
with open(os.path.join(DATA_DUMP, 'limit_new_1_1305.pickle'), 'wb') as handle:
    pickle.dump(limit, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Trim ventilator data using the manual filters

In [ ]:
data_pars_measurements_trimmed = {}
data_pars_settings_trimmed = {}
data_pars_alarms_trimmed = {}

for case in cases:
    
    if case in limit:
        data_pars_measurements_trimmed[case] = data_pars_measurements[case][limit[case][0] : limit[case][1]]
        data_pars_settings_trimmed[case] = data_pars_settings[case][limit[case][0] : limit[case][1]]
        data_pars_alarms_trimmed[case] = data_pars_alarms[case][limit[case][0] : limit[case][1]]
        
    else:
        data_pars_measurements_trimmed[case] = data_pars_measurements[case]
        data_pars_settings_trimmed[case] = data_pars_settings[case]
        data_pars_alarms_trimmed[case] = data_pars_alarms[case]

#### Export trimmed and cleaned DataFrames

In [ ]:
%%time

with open(os.path.join(DATA_DUMP, 'data_pars_measurements_trimmed_new_1_1305.pickle'), 'wb') as handle:
    pickle.dump(data_pars_measurements_trimmed, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(os.path.join(DATA_DUMP, 'data_pars_settings_trimmed_new_1_1305.pickle'), 'wb') as handle:
    pickle.dump(data_pars_settings_trimmed, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(os.path.join(DATA_DUMP, 'data_pars_alarms_trimmed_new_1_1305.pickle'), 'wb') as handle:
    pickle.dump(data_pars_alarms_trimmed, handle, protocol=pickle.HIGHEST_PROTOCOL)

### 6. Exploratory analysis on ventilation modes in the trimmed recordings

In [ ]:
vent_modes_trimmed = {}
for case in data_pars_settings_trimmed:
    # Multiply by two to get the number of seconds
    vent_modes_trimmed[case] = data_pars_settings_trimmed[case]['Ventilator_mode'].value_counts() * 2
    
vent_modes_trimmed = DataFrame(vent_modes_trimmed).T
vent_modes_trimmed.replace(np.nan, 0, inplace = True)

In [ ]:
# Add the duration of the recordings

recording_duration_trimmed = {}

for case in data_pars_settings_trimmed:
    recording_duration_trimmed[case] = 2 * len(data_pars_settings_trimmed[case])

In [ ]:
vent_modes_trimmed['ventilation'] = vent_modes_trimmed['IPPV']  + vent_modes_trimmed['SIMV'] + \
                                    vent_modes_trimmed['PSV']   + vent_modes_trimmed['HFO'] + \
                                    vent_modes_trimmed['SIPPV'] + vent_modes_trimmed['SIMVPSV']

vent_modes_trimmed['noninvasive'] = vent_modes_trimmed['CPAP']  + vent_modes_trimmed['DUOPAP'] + \
                                    vent_modes_trimmed['NCPAP'] + vent_modes_trimmed['O2therapy']  

vent_modes_trimmed['total'] = Series(recording_duration_trimmed)

In [ ]:
# How long was VG turned on
VG = {}
for case in data_pars_settings_trimmed:
    VG[case] = sum(data_pars_settings_trimmed[case]['VG_state'] == 'on') * 2
        
VG = Series(VG)
vent_modes_trimmed['VG'] = VG

In [ ]:
vent_modes_trimmed.head()

In [ ]:
print('SIMV:', sum(vent_modes_trimmed['SIMV'] > 0))
print('SIPPV:', sum(vent_modes_trimmed['SIPPV'] > 0))
print('SIMVPSV:', sum(vent_modes_trimmed['SIMVPSV'] > 0))
print('PSV:', sum(vent_modes_trimmed['PSV'] > 0))
print('HFO:', sum(vent_modes_trimmed['HFO'] > 0))
print('IPPV:', sum(vent_modes_trimmed['IPPV'] > 0))
print('NCPAP:', sum(vent_modes_trimmed['NCPAP'] > 0))
print('CPAP:', sum(vent_modes_trimmed['CPAP'] > 0))
print('DUOPAP:', sum(vent_modes_trimmed['DUOPAP'] > 0))
print('O2therapy:', sum(vent_modes_trimmed['O2therapy'] > 0))
print('ventilation:', sum(vent_modes_trimmed['ventilation'] > 0))
print('VG on:', sum(vent_modes_trimmed['VG'] > 0))
print('noninvasive:', sum(vent_modes_trimmed['noninvasive'] > 0))
print('total:', len(vent_modes_trimmed))

In [ ]:
# How many seconds of each ventilation mode in total ?
total_duration_trimmed = DataFrame(vent_modes_trimmed.sum(axis = 0), columns = ['duration (seconds)'])
total_duration_trimmed

##### Export Dataframes containing ventilator modes to Excel files and pickle archives

In [ ]:
writer = pd.ExcelWriter(os.path.join(DIR_WRITE, 'ventilation_modes_trimmed_new_1_1305.xlsx'))
vent_modes_trimmed.to_excel(writer, 'vent_modes')
total_duration_trimmed.to_excel(writer, 'total_duration')
writer.close()

##### Export graphs with the ventilation  modes of the trimmed recordings ventilator modes

In [ ]:
%%time

mapping_vent_mode = {1: 'IPPV', 2: 'SIPPV', 3: 'SIMV', 4: 'SIMVPSV', 5: 'PSV', 
                     6: 'CPAP', 7: 'NCPAP', 8: 'DUOPAP', 9: 'HFO', 10: 'O2therapy'}
mapping_vent_mode_rev = {value:key for key, value in mapping_vent_mode.items()}
mapping_vent_mode_rev

mapping_VG = {11: 'off', 12: 'on'}
mapping_VG_rev = {value:key for key, value in mapping_VG.items()}
mapping_VG_rev

par1 = 'Ventilator_mode'
par2 = 'VG_state'
dim = 'cmH$_2$O'

filetype='jpg'
dpi=200

yticks = np.arange(1, 13)
yticklabels = ['IPPV', 'SIPPV','SIMV','SIMVPSV', 'PSV', 'CPAP', 'NCPAP', 'DUOPAP', 'HFO', 'O2therapy',
              'VG OFF', 'VG ON']

for case in cases:
    #print('Saving %s' % case)
    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.1)
    ax = fig.add_subplot(1, 1, 1)
        
    data = data_pars_settings[case][par1].replace(mapping_vent_mode_rev).dropna().astype('int')
    data.plot(ax = ax, label = par1, color='blue', x_compat = True)
        
    data_2 = data_pars_settings[case][par2].replace(mapping_VG_rev).dropna().astype('int')
    data_2.plot(ax = ax, label = par2, color='blue', linestyle='dashed', x_compat = True)
        
    ax.set_xlabel('Time', size = 14, color = 'black')
    ax.set_ylabel('', size = 14, color = 'black')
    ax.set_ylim(0,17)
    ax.set_yticks(yticks)
    ax.set_yticklabels(yticklabels)
    ax.set_title(case,  size = 14, color = 'black')
    ax.legend(ncols=2)
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    majorFmt = dates.DateFormatter('%H:%M')
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    ax.tick_params(which = 'both', labelsize=12)
    
    fig.savefig(os.path.join(DATA_DUMP, 'fabian_cases_new', case, f'{case}_trimmed_ventilation_mode.{filetype}'), 
        dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,);
    
    if case != 'AT001120':
        plt.close()